In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
from IPython.display import display,clear_output 
from ipywidgets import Button,RadioButtons,Text,FloatText,Dropdown,IntSlider,Output,Layout,HBox,VBox,Layout,BoundedFloatText
import plotly.graph_objects as go

In [2]:
data_train = pd.read_csv("data_train.csv")
price_train = pd.read_csv("price_train.csv")
train_with_price = pd.read_csv('train_with_price.csv')
cols = data_train.columns
X_train,X_test,y_train,y_test = train_test_split(data_train, price_train, test_size=0.2,shuffle=True)

In [3]:
model = lightgbm.LGBMRegressor()
model.fit(X_train,y_train)
y_pred_model = model.predict(X_test)

In [4]:
area = Dropdown( options=['Richmond upon Thames', 'Camden', 'Lewisham', 'Kensington and Chelsea', 'Westminster',
                          'Hammersmith and Fulham','Lambeth', 'Southwark', 'Haringey', 'Wandsworth', 'Tower Hamlets',
                          'Hackney', 'Barnet', 'Merton', 'Islington', 'Hounslow', 'Newham','Brent', 'Barking and Dagenham', 
                          'Ealing', 'Kingston upon Thames','Sutton', 'Enfield', 'Bexley', 'Hillingdon', 'Greenwich',
                          'Waltham Forest', 'Croydon', 'Redbridge', 'City of London','Bromley', 'Harrow', 'Havering'],
                 value='Richmond upon Thames', 
                 description='Area:') 

style = {'description_width': 'initial'}
room_type = RadioButtons(
    options=['Entire home/apt', 'Private room', 'Shared room'],
    value='Entire home/apt', 
    description='Тype of apartment:',
    style = style)

text = BoundedFloatText(description='Predicted price',value='0.0',style = style)

bedroom_quantity = IntSlider(description='Number of bedrooms', min=1, max=5,value=1,style=style)
bathroom_quantity = IntSlider(description='Number of bathrooms', min=1, max=5,value=1,style=style)
guest_quantity = IntSlider(description='Number of guests', min=1, max=12,value=2,style=style)

price = Button(description="Click to predict price", button_style='success',style = style) 

def on_button_clicked_price(b): 
    clear_output()
    cols = data_train.columns
    data_pred = pd.DataFrame(data = np.zeros((1,len(cols))),columns=cols)
    data_pred['bedrooms'] = bedroom_quantity.value
    data_pred['bathrooms'] = bathroom_quantity.value
    data_pred['accommodates'] = guest_quantity.value
    data_pred['ng_'+area.value] = 1
    data_pred['rt_'+room_type.value] = 1
    text.value = round(np.expm1(model.predict(data_pred)[0]),2)
    display(room_type,guest_quantity,bedroom_quantity,bathroom_quantity)
    display(area,price,text)
    display(price_distribution_current_area,hist_current_area)
    
price.on_click(on_button_clicked_price)


price_distribution_current_area = Button(description="Price distribution in this area", button_style='success',
                                        layout=Layout(width='30%', height='40px')) 
def update(b):
    clear_output()
    df = train_with_price[(train_with_price['neighbourhood_cleansed']==area.value)&(train_with_price['price']<=500)&
                           (train_with_price['accommodates'] == guest_quantity.value)& 
                           (train_with_price['room_type']==room_type.value)][['price','latitude','longitude']]

    fig = go.Figure(data=go.Scatter( x= df['latitude'],y = df['longitude'],mode='markers',marker=dict(
         size=16, color=df['price'], colorscale='Viridis', showscale=True)))
        
    fig.update_layout( title_text="Price distribution in selected area",xaxis_title="latitude", yaxis_title="longitude")
    display(room_type,guest_quantity,bedroom_quantity,bathroom_quantity)
    display(area,price,text)
    display(price_distribution_current_area,hist_current_area)
    fig.show()
        
price_distribution_current_area.on_click(update)

hist_current_area = Button(description="Price histogram in this area", button_style='success',
                                        layout=Layout(width='30%', height='40px')) 

def hist(b):
    clear_output()
    df = train_with_price[(train_with_price['neighbourhood_cleansed']==area.value)&
                           (train_with_price['accommodates'] == guest_quantity.value)& 
                           (train_with_price['room_type']==room_type.value)][['price','latitude','longitude']]
    fig = go.Figure(go.Histogram(x=df['price']))
    fig.update_layout( title_text="Price histogram in selected area",xaxis_title="price", yaxis_title="room quantity")
    display(room_type,guest_quantity,bedroom_quantity,bathroom_quantity)
    display(area,price,text)
    display(price_distribution_current_area,hist_current_area)
    fig.show()
        
hist_current_area.on_click(hist)
display(room_type,guest_quantity,bedroom_quantity,bathroom_quantity)
display(area,price,text)
display(price_distribution_current_area,hist_current_area)


RadioButtons(description='Тype of apartment:', options=('Entire home/apt', 'Private room', 'Shared room'), sty…

IntSlider(value=2, description='Number of guests', max=12, min=1, style=SliderStyle(description_width='initial…

IntSlider(value=1, description='Number of bedrooms', max=5, min=1, style=SliderStyle(description_width='initia…

IntSlider(value=1, description='Number of bathrooms', max=5, min=1, style=SliderStyle(description_width='initi…

Dropdown(description='Area:', options=('Richmond upon Thames', 'Camden', 'Lewisham', 'Kensington and Chelsea',…

Button(button_style='success', description='Click to predict price', style=ButtonStyle())

BoundedFloatText(value=0.0, description='Predicted price', style=DescriptionStyle(description_width='initial')…

Button(button_style='success', description='Price distribution in this area', layout=Layout(height='40px', wid…

Button(button_style='success', description='Price histogram in this area', layout=Layout(height='40px', width=…